<a href="https://colab.research.google.com/github/LisaFyi/lisa-s-portofollio/blob/main/Crawling_Youtube_Comment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**IMPORT LIBRARY**

In [1]:
# Import Library
import pandas as pd
from googleapiclient.discovery import build

**CRAWLING COMMENT**

In [2]:
def video_comments(video_id):
	# empty list for storing reply
	replies = []

	# creating youtube resource object
	youtube = build('youtube', 'v3', developerKey=api_key)

	# retrieve youtube video results
	video_response = youtube.commentThreads().list(part='snippet,replies', videoId=video_id).execute()

	# iterate video response
	while video_response:

		# extracting required info
		# from each result object
		for item in video_response['items']:

			# Extracting comments ()
			published = item['snippet']['topLevelComment']['snippet']['publishedAt']
			user = item['snippet']['topLevelComment']['snippet']['authorDisplayName']

			# Extracting comments
			comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
			likeCount = item['snippet']['topLevelComment']['snippet']['likeCount']

			replies.append([published, user, comment, likeCount])

			# counting number of reply of comment
			replycount = item['snippet']['totalReplyCount']

			# if reply is there
			if replycount>0:
				# iterate through all reply
				for reply in item['replies']['comments']:

					# Extract reply
					published = reply['snippet']['publishedAt']
					user = reply['snippet']['authorDisplayName']
					repl = reply['snippet']['textDisplay']
					likeCount = reply['snippet']['likeCount']

					# Store reply is list
					#replies.append(reply)
					replies.append([published, user, repl, likeCount])

			# print comment with list of reply
			#print(comment, replies, end = '\n\n')

			# empty reply list
			#replies = []

		# Again repeat
		if 'nextPageToken' in video_response:
			video_response = youtube.commentThreads().list(
					part = 'snippet,replies',
					pageToken = video_response['nextPageToken'],
					videoId = video_id
				).execute()
		else:
			break
	#endwhile
	return replies

**PROCESSING**

In [3]:
# Insert Your API KEY
api_key = '<<YOUR API KEY>>'

# Enter video id
# contoh url video = https://youtu.be/xDlaALV5j04?si=FgLH5fM-tva436xq
video_id = "xDlaALV5j04" #isikan dengan kode / ID video

# Call function
comments = video_comments(video_id)
df = pd.DataFrame(comments, columns=['publishedAt', 'authorDisplayName', 'textDisplay', 'likeCount'])
df

,publishedAt,authorDisplayName,textDisplay,likeCount
0,2024-05-01T10:31:27Z,@wienwien89,pantes saja bnyak orng kaya lebih memilih ke r...,4963
1,2024-05-01T10:34:32Z,@azizatuszidko4514,Dikasih contoh sama Pak elbepe,138
2,2024-05-01T10:36:35Z,@larouge609,Luar negri jg ga jauh beda,104
3,2024-05-01T10:39:46Z,@ApaLoe3,Ke 3 hadir,37
4,2024-05-01T10:40:29Z,@suryoongkowidjaja2173,RS Luar negeri juga di lihat dulu RS mana.. RS...,72
...,...,...,...,...
7359,2024-05-01T04:20:33Z,@irrirachannel,Firtssss,0
7360,2024-05-01T04:20:32Z,@dianrini1099,Taiwan hadir,1
7361,2024-05-01T04:52:49Z,@winarti0741,Chiayi hadir,1
7362,2024-05-01T04:20:30Z,@riarostianengsih5394,Gerrcep,1


**SAVE CRAWLING RESULT**

In [4]:
df.to_csv('youtube-comments.csv', index=False)